# CWAS-Plus tutorial

## Introduction

Welcome to the tutorial of CWAS-Plus.

- The noncoding genome contains regulatory elements that play a critical role in human development. Due to advancements in whole genome sequencing (WGS) technologies, we are now able to explore mutations within these regulatory elements.

- To perform a genome-wide evaluation of noncoding mutations using WGS data, an analytic framework that enables fast and easy integration of diverse functional annotations to the WGS data and empowers multiple testing comparisons is essential.

- Recently, we introduced category-wide association study (CWAS), a novel statistical framework to identify noncoding association from WGS data. This methods creates multiple categories based on genomic and functional annotations and conducts genetic association tests on categories.

- In this tutorial, we will introduce **CWAS-Plus**, a Python package that enhances functionality and usability of the CWAS framework. With CWAS-Plus, users can effectively detect noncoding associations and perform efficient multiple hypotheses testing.

**CWAS-Plus workflow**
1. Annotation & categorization
2. Burden test
3. Risk score analysis
4. Burden shift analysis
5. Find the number of effective tests
6. Detecting Association With Networks (DAWN) analysis

## CWAS-Plus requirements


### Install required tools

In [ ]:
import os
os.environ['R_HOME'] = '/Users/yjkim/miniconda3/envs/test/lib/R'

In [ ]:
import rpy2.rinterface_lib.callbacks
my_callback = lambda *args: None
rpy2.rinterface_lib.callbacks.consolewrite_warnerror = my_callback

In [ ]:
from rpy2.robjects.packages import importr
utils = importr('utils')

In [ ]:
utils.install_packages("remotes", repos="https://cloud.r-project.org", quietly=True)

In [ ]:
import rpy2.robjects as ro

# Install the Matrix package, specifically version 1.6-1
ro.r('install.packages("Matrix")')
ro.r('remotes::install_version("Matrix", version = "1.6-1")')

In [ ]:
utils.install_packages('survival', repos="https://cloud.r-project.org", quietly=True)

In [ ]:
import os

# Set the environment variable for the notebook session
os.environ['PKG_CPPFLAGS'] = '-DHAVE_WORKING_LOG1P'

In [ ]:
utils.install_packages('glmnet', repos="https://cloud.r-project.org", quietly=True)

### Create pseudo VEP for configuration process

- CWAS-Plus checks whether 'VEP' is installed for proper configuration. Hence, for quick skipping annotation process, we will create pseudo VEP.

Install tabix and copy tabix to /usr/local/bin/vep

In [ ]:
!sudo apt-get install tabix
!which tabix
!pip install zarr -q

# Create fake VEP as we are skipping annotation process, which uses VEP
!cp /usr/local/bin/tabix /usr/local/bin/vep

In [ ]:
# Import modules for result interpretation
import pandas as pd
import polars as pl
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import zarr

### Input data
CWAS-Plus requires a list of variants and a list of samples containing the phenotype information. Adjustment factors for each sample can also be used if provided.

For this tutorial, we provide example input data obtained from [An et al., (2018)](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6432922/).

In [ ]:
import os

# Change directory to the home directory
os.chdir(os.environ['HOME'])

In [ ]:
!git clone https://github.com/joonan-lab/cwas-input-example.git

In the example data, there are three inputs.

- **de_novo_variants.vcf**
  - A list of variants
- **samples.txt**
  - A list of samples containing two columns, *SAMPLE* and *PHENOTYPE*.
- **adj_factors.txt**
  - A list of adjustment factors containing two columns, *SAMPLE* and *AdjustFactor*.

In [ ]:
%cd cwas-input-example
!ls -lhtr

Check how the input files are formatted from below.

In [ ]:
vcf = pd.read_table('de_novo_variants.vcf')
vcf.head(5)

In [ ]:
sample_info = pd.read_table('samples.txt')
sample_info.head(5)

In [ ]:
# The file contains 'SAMPLE' and 'AdjustFactor' column. The 'AdjustFactor' will be used to adjust the number of variants for each sample.
adj_factor = pd.read_table('adj_factors.txt')
adj_factor.head(5)

### Annotation datasets

CWAS-Plus creates a category with a combination of five domains.

- **Variant type**
  - Variant type refers to the type of changes of the variant.
- **GENCODE**
  - GENCODE is gene definition based on the location of the variant relative to genes. Domain includes coding domain, such as protein-truncating variants (PTVs), frameshift indels, missense, damaging missense, in frame indels, silent variants and noncoding domain, such as composed of promoter, UTR, intergenic, intron, long noncoding RNA, splice site non-canonical, and others.
- **Gene set**
  - Gene set domain comprises lists of disease-relevant genes. The dataset should be provided in text format.
- **Functional annotation**
  - Functional annotation is a domain for genomic intervals representing functions. For this domain, regions associated with epigenetic modifications or regulatory elements can be utilized. The datasets should be provided in bed format.
- **Functional score**
  - Functional scores represent regions score metrics related to specific genomic features, such as conservation or pathogenicity. This domain use regions that belong to specific score intervals. This information is processed binary, whether the region is above the cutoff or not. The datasets should be provided in bed format.

Users can provide customized annotation datasets to create categories. Each category serves as a hypothesis for conducting association tests, thereby enabling users to curate their own datasets tailored to the specific associations they wish to test.

For this tutorial, download the example set.

In [ ]:
os.chdir(os.environ['HOME'])
!git clone https://github.com/randrover/cwas_annotation_example.git
%cd cwas_annotation_example
!mv vep .vep

The example set contains two types of data.

- File for configuration
  - **configuration.txt**
- Files for creating categories
  - **annotation_keys.yaml**
  - **gene_matrix.txt**
  - **annotation_keys.yaml**
  - **BED files**


The **configuration.txt** file is for setting the environmental variables for CWAS-Plus. It contains the location of annotation datasets and VEP resources.

After installing CWAS-Plus, a file with the same name will be created in the working directory. We provide this file in the example set for user's convenience.
We will copy this file to the working directory after CWAS-Plus installation.

In [ ]:
!cat configuration.txt

Change the PATH of `ANNOTATION_DATA_DIR=/content/cwas_annotation_example` to the exact PATH of `cwas_annotation_example`. (This is required for configuration)

The **annotation_keys.yaml** contains the information of datasets for `functional annotation` and `functional scor`e. File are listed with the name that represent them in further analyses.

In [ ]:
!cat annotation_keys.yaml | head -6

The **gene_matrix.txt** contains a list of gene sets. This gene sets will be contained under the `gene set` domain.

In [ ]:
# The file contains columns representing each gene set used for gene set domain. The values in columns indicate whether the gene belong to the gene set (1) or not (0).
gene_matrix = pd.read_table('gene_matrix.txt')
gene_matrix.head(5)

Bed files that are listed in **annotation_keys.yaml** are also in this directory.

However, as we are skipping the annotation process, we are using fake files that have no content. This will prevent errors while CWAS-Plus checks whether the files exist.

In [ ]:
os.chdir(os.environ['HOME'])
%cd cwas_annotation_example
!ls *bed.gz

## Install CWAS-Plus

In [ ]:
!pip install cwas

Install package through `pip install`.

In [ ]:
os.chdir(os.environ['HOME'])
# '-w' option specify the path of the working directory of CWAS-Plus. Symlinks of annotation datsets will be stored here.
!cwas start -w .cwas

In [ ]:
# Copy the configuration file (that is already filled) to the working directory
!cp cwas_annotation_example/configuration.txt .cwas

Now start configuration. Users can force to overwrite with `-f` option.

In [ ]:
!cwas configuration -f

Also create output directory to save the results.

In [ ]:
os.chdir(os.environ['HOME'])
!mkdir cwas_output

## 1. Annotation & categorization

Now we are starting CWAS-Plus anlayses.

First of all, variants are annotated and categorized to create categories based on genomic and functional annotations. We will skip the **annotation** process and load annotated variants.

In [ ]:
%cd cwas_output
!wget -O de_novo_variants.annotated.vcf.gz "https://www.dropbox.com/scl/fi/n7zjumfpyxevk893jyqax/de_novo_variants.annotated.vcf.gz?rlkey=5mo92pjzdut1xvmcd0m5mdp1t&st=gfuswfn3&dl=0"

In [ ]:
# Annotations are in the INFO field.
annotated_vcf = pd.read_table('de_novo_variants.annotated.vcf.gz', skiprows = 10)
annotated_vcf.head(5)

In [ ]:
annotated_vcf['INFO'][1]

In **categorization**, we will create categories based on five domains and allocate variants to each category.

In [ ]:
os.chdir(os.environ['HOME'])
# Users can specify the number of worker processes to use with '-p' option. This option is constantly used in further analyses.
!cwas categorization -i cwas_output/de_novo_variants.annotated.vcf.gz -o_dir cwas_output/ -p 2

The index `SAMPLE` contains sample IDs. Other columns represent each category and the number of variants each sample has within the category.

In [ ]:
root = zarr.open('cwas_output/de_novo_variants.categorization_result.zarr', mode = 'r')
categorized = pd.DataFrame(root['data'],
                           index=root['metadata'].attrs['sample_id'],
                           columns=root['metadata'].attrs['category'])
categorized.index.name = 'SAMPLE'

In [ ]:
categorized.iloc[0:5,0:5]

## Burden test

In burden test, CWAS-Plus conducts association tests on eacy category.

### Binomial test

We will apply binomial testing to calculate significance.

In [ ]:
!cwas binomial_test -i cwas_output/de_novo_variants.categorization_result.zarr \
-o_dir cwas_output \
-s cwas-input-example/samples.txt \
-a cwas-input-example/adj_factors.txt

**de_novo_variants.burden_test.txt** contains the burden test results. Here, we have the relative risk and p-value for each category.

Relative risk>1 indicates that the category has risk of the disease phenotype.

In [ ]:
# 'P' for two-sided binomial p-value
# 'P_1side' for one-sided binomial p-value with alternative hypothesis 'greater.'
burden_result = pd.read_table('cwas_output/de_novo_variants.burden_test.txt')
burden_result.head(5)

**de_novo_variants.category_counts.txt** contains the number of variants in each category. Unlike burden test results, this file has unadjusted counts.

In [ ]:
counts = pd.read_table('cwas_output/de_novo_variants.category_counts.txt')
counts.head(5)

**de_novo_variants.category_info.txt** contains the information of each category.

In [ ]:
cat_info = pd.read_table('cwas_output/de_novo_variants.category_info.txt')
cat_info.head(5)

The burden test results are also displayed in volcano plots.

In [ ]:
count_thres = 7
print(f"Volcano plot will display categories with at least {count_thres} counts")

selected_categories = counts.loc[counts['Raw_counts'] >= count_thres, 'Category']
burden_result = burden_result[burden_result['Category'].isin(selected_categories)]

burden_result['-log_P'] = -np.log10(burden_result['P'])
burden_result['log2_RR'] = np.log2(burden_result['Relative_Risk'])

max_rr = max(burden_result.loc[burden_result.log2_RR!=np.inf, 'log2_RR'])
min_rr = min(burden_result.loc[burden_result.log2_RR!=-np.inf, 'log2_RR'])
max_val = max(abs(max_rr),abs(min_rr))
max_x = np.trunc(max_val) + 2

In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(x='log2_RR', y='-log_P', data=burden_result, s=5, color = 'grey')
plt.axhline(y=-np.log10(0.05), color='r', linestyle='--', label='-log10(0.05)')
plt.axvline(x=0, color='grey', linestyle='-')
plt.xlabel('Relative risk (log2)')
plt.ylabel('P (-log10)')
plt.xlim(-(max_x), (max_x))
plt.title('Burden test results')
plt.legend()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Define threshold for category selection
count_thres = 7
print(f"Volcano plot will display categories with at least {count_thres} counts")

# Filter categories
selected_categories = counts.loc[counts['Raw_counts'] >= count_thres, 'Category']
burden_result = burden_result[burden_result['Category'].isin(selected_categories)]

# Merge with cat_info to get is_coding and is_noncoding
burden_result = burden_result.merge(cat_info[['Category', 'is_coding', 'is_noncoding']], on='Category', how='left')

# Compute log-transformed values
burden_result['-log_P'] = -np.log10(burden_result['P'])
burden_result['log2_RR'] = np.log2(burden_result['Relative_Risk'])

# Compute Bonferroni correction
total_tests = len(burden_result)
bonferroni_cutoff = 0.05 / total_tests
bonferroni_threshold = -np.log10(bonferroni_cutoff)

# Define x-axis limits
max_rr = max(burden_result.loc[burden_result.log2_RR != np.inf, 'log2_RR'])
min_rr = min(burden_result.loc[burden_result.log2_RR != -np.inf, 'log2_RR'])
max_val = max(abs(max_rr), abs(min_rr))
max_x = np.trunc(max_val) + 2

# Assign colors based on category type
def assign_color(row):
    if row['is_coding'] == 1:
        return 'blue'  # Coding variants
    elif row['is_noncoding'] == 1:
        return 'red'  # Non-coding variants
    else:
        return 'grey'  # Other categories

burden_result['color'] = burden_result.apply(assign_color, axis=1)

# Create volcano plot
plt.figure(figsize=(8, 6))
sns.scatterplot(x='log2_RR', y='-log_P', data=burden_result, hue='color', palette={'blue': 'blue', 'red': 'red', 'grey': 'grey'}, s=5)

# Add significance cutoffs
plt.axhline(y=-np.log10(0.05), color='r', linestyle='--', label='-log10(0.05)')
plt.axhline(y=bonferroni_threshold, color='blue', linestyle='--', label=f'Bonferroni ({total_tests} tests)')

# Add vertical line at log2(RR) = 0
plt.axvline(x=0, color='grey', linestyle='-')

# Labels and title
plt.xlabel('Relative risk (log2)')
plt.ylabel('P (-log10)')
plt.xlim(-max_x, max_x)
plt.title('Burden test results')

# Legend
plt.legend(title="Category Type", labels=["is_coding (blue)", "is_noncoding (red)", "Others (grey)"])

plt.show()


### Permutation test
We can also conduct permutations to obtain p-values. In each permutation, CWAS-Plus randomly assigns phenotypes to samples.

The outputs are used in burden shift analysis and DAWN analysis.

In [ ]:
# Specify the number of permutations with '-n' option. For quick run, we used 15 permutations. We recommend 10,000 in real analyses.
!cwas permutation_test -i cwas_output/de_novo_variants.categorization_result.zarr \
-o_dir cwas_output \
-s cwas-input-example/samples.txt \
-a cwas-input-example/adj_factors.txt \
-n 10 \
-b \
-p 1

The output contains the p-value calculated by comparing the binomial p-value with the p-values from permutations. This will be used in DAWN analysis.

In [ ]:
permute_results = pd.read_table('cwas_output/de_novo_variants.permutation_test.txt.gz')
permute_results.head(5)

Binomial p-values from each permutation are saved in `de_novo_variants.binom_pvals.txt.gz`. This will be used in burden shift analysis.

In [ ]:
# This file is saved with 'This file is saved with '-b' option.
binom_pvals = pl.read_csv('cwas_output/de_novo_variants.binom_pvals.txt.gz', separator = '\t')
binom_pvals[0:5,0:5]

## Risk score analysis

In risk score analysis, we will identify categories that can predict the disease phenotype.

Specifically, CWAS-Plus trains a *Lasso regression model* and find categories that contribute to the prediction. We will use 80% of samples as training set and the remaining samples as test set.

#### Install glmnet package in R

In [ ]:
!cwas risk_score --help

In [ ]:
# Specify the number of permutations with '-n' option. For quick run, we used 10. We recommend doing at least 1,000 permutations in real analysis.
# Specify the number of regressions with '-n_reg' option.
!cwas risk_score -i cwas_output/de_novo_variants.categorization_result.zarr \
-o_dir cwas_output \
-s cwas-input-example/samples.txt \
-a cwas-input-example/adj_factors.txt \
--domain_list noncoding \
-c cwas_output/de_novo_variants.category_info.txt \
-thr 3 \
-tf 0.8 \
-n_reg 10 \
-f 5 \
-n 10 \
-p 2

In the results, the first row represents the average R2 of five regressions. The p-value (`perm_P`) is calculated using this average R2.

The remaining rows represent R2s from each regression.

In [ ]:
%cd cwas_output
!ls -lhtr

In [ ]:
os.chdir(os.environ['HOME'])
# 'N_select' indicates the number of categories that contribute to phenotype prediction.
risk_score_result = pd.read_table('cwas_output/de_novo_variants.lasso_results_thres_3.noncoding.txt')
risk_score_result

**de_novo_variants.lasso_coef_thres_3.txt** contains categories that contribute to the phenotype prediction. Values in each row indicate the coefficient regression models. Positive large coefficient means strong positive effect on the phenotype prediction when the number of variants increases.

In [ ]:
risk_score_cat = pd.read_table('cwas_output/de_novo_variants.lasso_coef_thres_3.noncoding.txt')
risk_score_cat

Let's look what kind of categories contribute to the prediction.

In [ ]:
filt_cats = risk_score_cat.columns[1:].tolist()
risk_cat_info = cat_info[cat_info['Category'].isin(filt_cats)]
risk_cat_info = risk_cat_info[risk_cat_info['is_noncoding']==1]

Categories with EncodeTFBS account for some proportion in noncoding categories that contribute to the prediction.

In [ ]:
risk_cat_info['functional_annotation'].value_counts()

## Burden shift analysis

In burden shift analysis, we will find domains that are enriched in cases. CWAS-Plus tests enrichment in a single domain and combinations of two domains. From the results, we can find which domains are candidates of containing risk.

In [ ]:
!cwas burden_shift -i cwas_output/de_novo_variants.burden_test.txt \
-b cwas_output/de_novo_variants.binom_pvals.txt.gz \
-o_dir cwas_output \
-c_info cwas_output/de_novo_variants.category_info.txt \
-c_count cwas_output/de_novo_variants.category_counts.txt \
-c_cutoff 7 \
--pval 0.05 \
-t test

Explore the result plots generated as outputs.

In [ ]:
burdenshift = pd.read_table('cwas_output/de_novo_variants.burdenshift_p0.05_cutoff7.test.txt')
burdenshift = burdenshift[burdenshift['P_case']<0.05]
burdenshift = burdenshift[burdenshift['N_cats_case']>=10]
burdenshift.sort_values(by=['P_case', 'N_cats_case'])

Categories with EncodeTFBS shows high enrichment in cases. This can be a candidate domain associated to ASD risk.

In [ ]:
subset_df = burdenshift[burdenshift['Category_set'].str.contains('CHD8Common', case=False)]
subset_df

## Find the number of effective tests

CWAS-Plus provides a unique method to assess the noncoding association. With calculating the number of effective tests, users can identify accurate noncoding associations with multiple comparisons.

To find the number of effective tests, we will use the correlation values between categories and count the number of leading eigenvalues that account for more than 99% of the total variations.

As calculating correlation values take time, we will load the data for quick run.

Find the number of effective tests from the correlation values.

In [ ]:
!cwas correlation -i cwas_output/de_novo_variants.categorization_result.zarr \
-v cwas_output/de_novo_variants.annotated.vcf.gz \
-o_dir cwas_output \
-c_info cwas_output/de_novo_variants.category_info.txt \
-p 8 -cm variant -im

In [ ]:
!cwas effective_num_test -i cwas_output/de_novo_variants.correlation_matrix.zarr \
-o_dir cwas_output \
-ef -thr 8 -if corr -n 10000 \
-c_count cwas_output/de_novo_variants.category_counts.txt

The number of effective tests is **2,665**.

## DAWN analysis
In DAWN analysis, we will find novel risk domains and investigate subnetworks of risk domains within categories of risk domain.

First, we will generate inputs (eigen vectors) for DAWN analysis from correlation values bewteen categories. We will only use noncoding categories of EncodeTFBS domain.

In [ ]:
# Filter noncoding categories within EncodeTFBS region.
cat_info = pd.read_table('cwas_output/de_novo_variants.category_info.txt')
counts = pd.read_table('cwas_output/de_novo_variants.category_counts.txt')
cat_info = cat_info[cat_info['Category'].isin(counts[counts['Raw_counts']>=8]['Category'])]
cat_info = cat_info[cat_info['is_noncoding']==1]
cat_info = cat_info[cat_info['gene_set']=='LOEUF37']
cat_info.to_csv('cwas_output/de_novo_variants.subset_categories.txt', sep='\t', index=False)
cat_info

In [ ]:
!cwas effective_num_test -i cwas_output/de_novo_variants.correlation_matrix.zarr \
-o_dir cwas_output -if corr -n 10000 \
-thr 8 \
-t CHD8 \
-c_set cwas_output/de_novo_variants.subset_categories.txt \
-c_count cwas_output/de_novo_variants.category_counts.txt

Then, with the generated inputs, we can construct a network of these categories and assess the disease association of them.

In [ ]:
!cwas dawn -e cwas_output/de_novo_variants.eig_vecs.CHD8.zarr \
-c cwas_output/de_novo_variants.correlation_matrix.zarr \
-P cwas_output/de_novo_variants.permutation_test.txt.gz \
-o_dir cwas_output \
-r 2,100 \
-s 2025 \
-t TFBS.exact \
-T exact \
-c_count cwas_output/de_novo_variants.category_counts.txt \
-C 20 \
-R 0.12 \
-S 2 \
-p 2

Check the generated plot and explore which clsuters are connected and show disease association.

In [ ]:
# 'Pval.cluster' indicates the p-value of a cluster.
# 'Risk' indicates the relative risk of a cluster. Risk>1 means that the cluster has disease association.
dawn_result = pd.read_csv('cwas_output/TFBS.exact.ipvalue_fdr_ipvalue_risk.csv')
dawn_result.sort_values(by=['Pval.cluster'])

Check which categories are included in the cluster with risk. For example, cluster 81 can be represented as intergenic cluster. The results inducate that intergenic variants carry risk within network of EncodeTFBS domain.

In [ ]:
cluster_info = pd.read_csv('cwas_output/TFBS.exact.cluster_annotation.csv')
cluster_info['81']